![cover](complements/cover.jpg)

# Module Import

In [10]:
import torch
from torchvision import transforms
from dataset import DatasetFromFolder
from model import Generator, Discriminator

import utils
import argparse
import os, itertools
from logger import Logger
import numpy as np

# Argument Parsing & Directories Initialization

In [6]:
parser = argparse.ArgumentParser()

#Data Set Parameter
parser.add_argument('--dataset', required=False, default='horse2zebra', help='input dataset')
parser.add_argument('--batch_size', type=int, default=1, help='train batch size')
parser.add_argument('--input_size', type=int, default=256, help='input size')
parser.add_argument('--resize_scale', type=int, default=286, help='resize scale (0 is false)')
parser.add_argument('--crop_size', type=int, default=256, help='crop size (0 is false)')
parser.add_argument('--fliplr', type=bool, default=True, help='random fliplr True of False')

#Model Parameters 
parser.add_argument('--ngf', type=int, default=32) # number of generator filters
parser.add_argument('--ndf', type=int, default=64) # number of discriminator filters
parser.add_argument('--num_resnet', type=int, default=6, help='number of resnet blocks in generator')

#Learning Parameters
parser.add_argument('--num_epochs', type=int, default=30, help='number of train epochs')
parser.add_argument('--decay_epoch', type=int, default=100, help='start decaying learning rate after this number')
parser.add_argument('--lrG', type=float, default=0.0002, help='learning rate for generator, default=0.0002')
parser.add_argument('--lrD', type=float, default=0.0002, help='learning rate for discriminator, default=0.0002')
parser.add_argument('--beta1', type=float, default=0.5, help='beta1 for Adam optimizer')
parser.add_argument('--beta2', type=float, default=0.999, help='beta2 for Adam optimizer')
parser.add_argument('--lambdaA', type=float, default=10, help='lambdaA for cycle loss')
parser.add_argument('--lambdaB', type=float, default=10, help='lambdaB for cycle loss')
params = parser.parse_args([])
print(params)

# Directories for loading data and saving results
data_dir = 'data/' + params.dataset + '/'
save_dir = params.dataset + '_results/'
model_dir = params.dataset + '_model/'

Namespace(batch_size=1, beta1=0.5, beta2=0.999, crop_size=256, dataset='horse2zebra', decay_epoch=100, fliplr=True, input_size=256, lambdaA=10, lambdaB=10, lrD=0.0002, lrG=0.0002, ndf=64, ngf=32, num_epochs=1, num_resnet=6, resize_scale=286)


In [3]:
if not os.path.exists(data_dir) : 
    os.makedirs(data_dir)
    file = 'ZIP_FILE.zip'
    url = 'https://people.eecs.berkeley.edu/~taesung_park/CycleGAN/datasets/' + params.dataset + '.zip'
    print(url)
    !wget -N $url -O $file
    !unzip $file -d 'data/'
    !rm $file
if not os.path.exists(save_dir):
    os.mkdir(save_dir)
if not os.path.exists(model_dir):
    os.mkdir(model_dir)

# Set the logger
D_A_log_dir = save_dir + 'D_A_logs'
D_B_log_dir = save_dir + 'D_B_logs'
if not os.path.exists(D_A_log_dir):
    os.mkdir(D_A_log_dir)
D_A_logger = Logger(D_A_log_dir)
if not os.path.exists(D_B_log_dir):
    os.mkdir(D_B_log_dir)
D_B_logger = Logger(D_B_log_dir)

G_A_log_dir = save_dir + 'G_A_logs'
G_B_log_dir = save_dir + 'G_B_logs'
if not os.path.exists(G_A_log_dir):
    os.mkdir(G_A_log_dir)
G_A_logger = Logger(G_A_log_dir)
if not os.path.exists(G_B_log_dir):
    os.mkdir(G_B_log_dir)
G_B_logger = Logger(G_B_log_dir)

cycle_A_log_dir = save_dir + 'cycle_A_logs'
cycle_B_log_dir = save_dir + 'cycle_B_logs'
if not os.path.exists(cycle_A_log_dir):
    os.mkdir(cycle_A_log_dir)
cycle_A_logger = Logger(cycle_A_log_dir)
if not os.path.exists(cycle_B_log_dir):
    os.mkdir(cycle_B_log_dir)
cycle_B_logger = Logger(cycle_B_log_dir)

img_log_dir = save_dir + 'img_logs'
if not os.path.exists(img_log_dir):
    os.mkdir(img_log_dir)
img_logger = Logger(img_log_dir)

# Datasets

In [4]:
# Data pre-processing
transform = transforms.Compose([transforms.Resize(params.input_size),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))])

# Train data
train_data_A = DatasetFromFolder(data_dir, subfolder='trainA', transform=transform,
                                 resize_scale=params.resize_scale, crop_size=params.crop_size, fliplr=params.fliplr)
train_data_loader_A = torch.utils.data.DataLoader(dataset=train_data_A,
                                                  batch_size=params.batch_size,
                                                  shuffle=True)
train_data_B = DatasetFromFolder(data_dir, subfolder='trainB', transform=transform,
                                 resize_scale=params.resize_scale, crop_size=params.crop_size, fliplr=params.fliplr)
train_data_loader_B = torch.utils.data.DataLoader(dataset=train_data_B,
                                                  batch_size=params.batch_size,
                                                  shuffle=True)

# ------------------------------------------------------------------------------------------
# Test data
test_data_A = DatasetFromFolder(data_dir, subfolder='testA', transform=transform)
test_data_loader_A = torch.utils.data.DataLoader(dataset=test_data_A,
                                                 batch_size=params.batch_size,
                                                 shuffle=False)
test_data_B = DatasetFromFolder(data_dir, subfolder='testB', transform=transform)
test_data_loader_B = torch.utils.data.DataLoader(dataset=test_data_B,
                                                 batch_size=params.batch_size,
                                                 shuffle=False)

# Get specific test images
test_real_A_data = train_data_A.__getitem__(11).unsqueeze(0)  # Convert to 4d tensor (BxNxHxW)
test_real_B_data = train_data_B.__getitem__(91).unsqueeze(0)

# Models & Optimizers & Criterions

In [5]:
# Models
G_A = Generator(3, params.ngf, 3, params.num_resnet) # arguments : input_dim, num_filter, output_dim, num_resnet
G_B = Generator(3, params.ngf, 3, params.num_resnet)
D_A = Discriminator(3, params.ndf, 1)               # arguments : input_dim, num_filter, output_dim
D_B = Discriminator(3, params.ndf, 1) 

G_A.normal_weight_init(mean=0.0, std=0.02)
G_B.normal_weight_init(mean=0.0, std=0.02)
D_A.normal_weight_init(mean=0.0, std=0.02)
D_B.normal_weight_init(mean=0.0, std=0.02)

G_A.cuda()
G_B.cuda()
D_A.cuda()
D_B.cuda()

# optimizers
G_optimizer = torch.optim.Adam(itertools.chain(G_A.parameters(), G_B.parameters()), lr=params.lrG, betas=(params.beta1, params.beta2))
D_A_optimizer = torch.optim.Adam(D_A.parameters(), lr=params.lrD, betas=(params.beta1, params.beta2))
D_B_optimizer = torch.optim.Adam(D_B.parameters(), lr=params.lrD, betas=(params.beta1, params.beta2))

# Loss function
MSE_loss = torch.nn.MSELoss().cuda()
L1_loss = torch.nn.L1Loss().cuda()

# # Training GAN
D_A_avg_losses = []
D_B_avg_losses = []
G_A_avg_losses = []
G_B_avg_losses = []
cycle_A_avg_losses = []
cycle_B_avg_losses = []

# Generated image pool
num_pool = 50
fake_A_pool = utils.ImagePool(num_pool)
fake_B_pool = utils.ImagePool(num_pool)

# Training
<br><br>
## Concepts
![concept](complements/concept.jpg)
<br>
## GAN Losses
![gan_loss](complements/gan_loss.JPG)
<br>
## Cycle Consistency Losses
![cycle_consistency_loss](complements/cycle_consistency.JPG)
<br>
## Full Objectives = Gan Losses + Cycle Consistency Losses 
![full_objectives](complements/full_objectives.JPG)

In [8]:
step = 0
for epoch in range(params.num_epochs):
    D_A_losses = []
    D_B_losses = []
    G_A_losses = []
    G_B_losses = []
    cycle_A_losses = []
    cycle_B_losses = []

    # learning rate decay
    if (epoch + 1) > params.decay_epoch:
        D_A_optimizer.param_groups[0]['lr'] -= params.lrD / (params.num_epochs - params.decay_epoch)
        D_B_optimizer.param_groups[0]['lr'] -= params.lrD / (params.num_epochs - params.decay_epoch)
        G_optimizer.param_groups[0]['lr'] -= params.lrG / (params.num_epochs - params.decay_epoch)

    # training
    for i, (real_A, real_B) in enumerate(zip(train_data_loader_A, train_data_loader_B)):

        # input image data
        real_A = real_A.cuda()
        real_B = real_B.cuda()

        # ------------------------ Train generator G ----------------------------
        # A -> B
        fake_B = G_A(real_A)
        D_B_fake_decision = D_B(fake_B)
        G_A_loss = MSE_loss(D_B_fake_decision, torch.ones(D_B_fake_decision.size()).cuda())

        # forward cycle loss
        recon_A = G_B(fake_B)
        cycle_A_loss = L1_loss(recon_A, real_A) * params.lambdaA

        # B -> A
        fake_A = G_B(real_B)
        D_A_fake_decision = D_A(fake_A)
        G_B_loss = MSE_loss(D_A_fake_decision, torch.ones(D_A_fake_decision.size()).cuda())

        # backward cycle loss
        recon_B = G_A(fake_A)
        cycle_B_loss = L1_loss(recon_B, real_B) * params.lambdaB

        # Back propagation
        G_loss = G_A_loss + G_B_loss + cycle_A_loss + cycle_B_loss
        G_optimizer.zero_grad()
        G_loss.backward()
        G_optimizer.step()
        
        # ------------------------ Train discriminator D_A ------------------------
        D_A_real_decision = D_A(real_A)
        D_A_real_loss = MSE_loss(D_A_real_decision, torch.ones(D_A_real_decision.size()).cuda())
        
        fake_A = fake_A_pool.query(fake_A)
        
        D_A_fake_decision = D_A(fake_A)
        D_A_fake_loss = MSE_loss(D_A_fake_decision, torch.zeros(D_A_fake_decision.size()).cuda())

        # Back propagation
        D_A_loss = (D_A_real_loss + D_A_fake_loss) * 0.5
        D_A_optimizer.zero_grad()
        D_A_loss.backward()
        D_A_optimizer.step()

        # ------------------------ Train discriminator D_B ------------------------
        D_B_real_decision = D_B(real_B)
        D_B_real_loss = MSE_loss(D_B_real_decision, torch.ones(D_B_real_decision.size()).cuda())
        fake_B = fake_B_pool.query(fake_B)
        D_B_fake_decision = D_B(fake_B)
        D_B_fake_loss = MSE_loss(D_B_fake_decision, torch.zeros(D_B_fake_decision.size()).cuda())

        # Back propagation
        D_B_loss = (D_B_real_loss + D_B_fake_loss) * 0.5
        D_B_optimizer.zero_grad()
        D_B_loss.backward()
        D_B_optimizer.step()

        # ------------------------ Print -----------------------------
        # loss values
        D_A_losses.append(D_A_loss.item())
        D_B_losses.append(D_B_loss.item())
        G_A_losses.append(G_A_loss.item())
        G_B_losses.append(G_B_loss.item())
        cycle_A_losses.append(cycle_A_loss.item())
        cycle_B_losses.append(cycle_B_loss.item())

        if i%10 == 0:
            print('Epoch [%d/%d], Step [%d/%d], D_A_loss: %.4f, D_B_loss: %.4f, G_A_loss: %.4f, G_B_loss: %.4f'
                  % (epoch+1, params.num_epochs, i+1, len(train_data_loader_A), D_A_loss.item(), D_B_loss.item(), G_A_loss.item(), G_B_loss.item()))

        # ============ TensorBoard logging ============#
        D_A_logger.scalar_summary('losses', D_A_loss.item(), step + 1)
        D_B_logger.scalar_summary('losses', D_B_loss.item(), step + 1)
        G_A_logger.scalar_summary('losses', G_A_loss.item(), step + 1)
        G_B_logger.scalar_summary('losses', G_B_loss.item(), step + 1)
        cycle_A_logger.scalar_summary('losses', cycle_A_loss.item(), step + 1)
        cycle_B_logger.scalar_summary('losses', cycle_B_loss.item(), step + 1)
        step += 1

    D_A_avg_loss = torch.mean(torch.FloatTensor(D_A_losses))
    D_B_avg_loss = torch.mean(torch.FloatTensor(D_B_losses))
    G_A_avg_loss = torch.mean(torch.FloatTensor(G_A_losses))
    G_B_avg_loss = torch.mean(torch.FloatTensor(G_B_losses))
    cycle_A_avg_loss = torch.mean(torch.FloatTensor(cycle_A_losses))
    cycle_B_avg_loss = torch.mean(torch.FloatTensor(cycle_B_losses))

    # avg loss values for plot
    D_A_avg_losses.append(D_A_avg_loss)
    D_B_avg_losses.append(D_B_avg_loss)
    G_A_avg_losses.append(G_A_avg_loss)
    G_B_avg_losses.append(G_B_avg_loss)
    cycle_A_avg_losses.append(cycle_A_avg_loss)
    cycle_B_avg_losses.append(cycle_B_avg_loss)

    # Show result for test image
    test_real_A = test_real_A_data.cuda()
    test_fake_B = G_A(test_real_A)
    test_recon_A = G_B(test_fake_B)

    test_real_B = test_real_B_data.cuda()
    test_fake_A = G_B(test_real_B)
    test_recon_B = G_A(test_fake_A)

    utils.plot_train_result([test_real_A, test_real_B], [test_fake_B, test_fake_A], [test_recon_A, test_recon_B],
                            epoch, save=True, save_dir=save_dir)

    # log the images
    result_AtoB = np.concatenate((utils.to_np(test_real_A), utils.to_np(test_fake_B), utils.to_np(test_recon_A)), axis=3)
    result_BtoA = np.concatenate((utils.to_np(test_real_B), utils.to_np(test_fake_A), utils.to_np(test_recon_B)), axis=3)

    info = { 'result_AtoB': result_AtoB.transpose(0, 2, 3, 1),  # convert to BxHxWxC
             'result_BtoA': result_BtoA.transpose(0, 2, 3, 1) }

    for tag, images in info.items():
        img_logger.image_summary(tag, images, epoch + 1)

Epoch [1/1], Step [1/1067], D_A_loss: 0.7572, D_B_loss: 0.5360, G_A_loss: 0.9199, G_B_loss: 1.3571
Epoch [1/1], Step [11/1067], D_A_loss: 0.2966, D_B_loss: 0.4281, G_A_loss: 0.4686, G_B_loss: 0.3903
Epoch [1/1], Step [21/1067], D_A_loss: 0.3075, D_B_loss: 0.2699, G_A_loss: 0.3335, G_B_loss: 0.3478
Epoch [1/1], Step [31/1067], D_A_loss: 0.2117, D_B_loss: 0.2828, G_A_loss: 0.3091, G_B_loss: 0.3918
Epoch [1/1], Step [41/1067], D_A_loss: 0.2810, D_B_loss: 0.1995, G_A_loss: 0.3551, G_B_loss: 0.2603
Epoch [1/1], Step [51/1067], D_A_loss: 0.2599, D_B_loss: 0.2508, G_A_loss: 0.3079, G_B_loss: 0.4972
Epoch [1/1], Step [61/1067], D_A_loss: 0.1657, D_B_loss: 0.3117, G_A_loss: 0.1817, G_B_loss: 0.3903
Epoch [1/1], Step [71/1067], D_A_loss: 0.2893, D_B_loss: 0.2302, G_A_loss: 0.3285, G_B_loss: 0.2182
Epoch [1/1], Step [81/1067], D_A_loss: 0.1836, D_B_loss: 0.2592, G_A_loss: 0.3782, G_B_loss: 0.2973
Epoch [1/1], Step [91/1067], D_A_loss: 0.2448, D_B_loss: 0.1890, G_A_loss: 0.4060, G_B_loss: 0.2271
E

# Save Results

In [9]:
# Plot average losses
avg_losses = []
avg_losses.append(D_A_avg_losses)
avg_losses.append(D_B_avg_losses)
avg_losses.append(G_A_avg_losses)
avg_losses.append(G_B_avg_losses)
avg_losses.append(cycle_A_avg_losses)
avg_losses.append(cycle_B_avg_losses)
utils.plot_loss(avg_losses, params.num_epochs, save=True, save_dir=save_dir)

# Make gif
utils.make_gif(params.dataset, params.num_epochs, save_dir=save_dir)
# Save trained parameters of model
torch.save(G_A.state_dict(), model_dir + 'generator_A_param.pkl')
torch.save(G_B.state_dict(), model_dir + 'generator_B_param.pkl')
torch.save(D_A.state_dict(), model_dir + 'discriminator_A_param.pkl')
torch.save(D_B.state_dict(), model_dir + 'discriminator_B_param.pkl')